# Pangolin SARS-CoV-2 Pipeline Notebook

## Overview 
SARS-CoV-2 sequence is usually analyzed using a bioinformatic pipeline called Pangolin. Here we will download some genomic data and run Pangolin following [standard instructions](https://cov-lineages.org/resources/pangolin/usage.html). 

## Prerequisites
We assume you have access to Azure AI Studio and have already deployed an LLM 

## Learning objectives
+ Download genomic data from NCBI from the commnd line
+ Run pangolin to identify viral lineages
+ Generate a phylogeny to visualize lineage identity

## Get started

### Install packages

In [ ]:
#change this depending on how many threads are available in your notebook
CPU=4

In [ ]:
! curl -L -O https://github.com/conda-forge/miniforge/releases/latest/download/Mambaforge-$(uname)-$(uname -m).sh
! bash Mambaforge-$(uname)-$(uname -m).sh -u -b -p $HOME/mambaforge

In [ ]:
#add to your path
import os
os.environ["PATH"] += os.pathsep + os.environ["HOME"]+"/mambaforge/bin"

In [ ]:
#install biopython to import packages below
! pip install biopython==1.74

In [ ]:
! mamba install pangolin=4.1.1 -c conda-forge -c bioconda -y

In [ ]:
! mamba install ipyrad iqtree -c conda-forge -c bioconda -y

In [ ]:
#import libraries
import os
from Bio import SeqIO
from Bio import Entrez
import ipyrad as ipa
import toytree

### Set up directory structure

In [ ]:
if not os.path.exists('pangolin_analysis'):
    os.mkdir('pangolin_analysis')
os.chdir('pangolin_analysis')

In [ ]:
if os.path.exists('sarscov2_sequences.fasta'):
    os.remove('sarscov2_sequences.fasta')
!rm sarscov2_*
!rm lineage_report.csv

### Fetch viral sequences using a list of accession IDs

In [ ]:
#give a list of accession number for covid sequences
acc_nums=['NC_045512','LR757995','LR757996','OL698718','OL677199','OL672836','MZ914912','MZ916499','MZ908464','MW580573','MW580574','MW580576','MW991906','MW931310','MW932027','MW424864','MW453109','MW453110']
print('the number of sequences we will analyze = ',len(acc_nums))

Let this block run without going to the next until it finishes, otherwise you may get an error about too many requests. If that happens, reset your kernel and just rerun everything (except installing software).

In [ ]:
#use the bio.entrez toolkit within biopython to download the accession numbers
#save those sequences to a single fasta file
Entrez.email = "email@example.com"  # Always tell NCBI who you are
filename = "sarscov2_seqs.fasta"
if not os.path.isfile(filename):
    # Downloading...
    for acc in acc_nums:
        net_handle = Entrez.efetch(
            db="nucleotide", id=acc, rettype="fasta", retmode="text"
        )
        out_handle = open(filename, "a")
        out_handle.write(net_handle.read())
        out_handle.close()
        net_handle.close()
        print("Saved",acc)

In [ ]:
#make sure our fasta file has the same number of seqs as the acc_nums list
print('the number of seqs in our fasta file: ')
! grep '>' sarscov2_seqs.fasta | wc -l

In [ ]:
#let's peek at our new fasta file
! head sarscov2_seqs.fasta

### Run pangolin to identify lineages and output alignment
Here we call pangolin, give it our input sequences and the number of threads. We also tell it to output the alignment. The full list of pangolin parameters can be found in the [docs](https://cov-lineages.org/resources/pangolin/usage.html).

In [ ]:
! pangolin -h

In [ ]:
! pangolin sarscov2_seqs.fasta --alignment --threads $CPU

You can view the output file from pangolin called lineage_report.csv (within pangolin_analysis folder) by double clicking on the file, or by right clicking and downloading. What lineages are present in the dataset? Is Omicron in there?

### Run iqtree to estimate maximum likelihood tree for our sequences
iqtree can find the best nucleotide model for the data, but here we are going to assign a model to save time (HKY) and just estimate the phylogeny without any bootstrap support values. 

In [ ]:
#run iqtree with threads = $CPU variable, if you exclude the -m it will do a phylogenetic model search before tree search
! iqtree -s sequences.aln.fasta -nt $CPU -m HKY --prefix sarscov2_tree --redo-tree

### Visualize the tree with toytree

In [ ]:
#Define the tree file
tre = toytree.tree('sarscov2_tree.treefile')

In [ ]:
#draw the tree
rtre = tre.root(wildcard="OL")
rtre.draw(tip_labels_align=True);

You can also visualize the tree by downloading it and opening in figtree.

## Conclusions
Here you learned how to use Azure ML Studio to conduct a basic phylogenetic analysis

## Clean Up
Make sure you stop your compute instance and if desired, delete the resource group associated with this tutorial.